# *labrary usage and prametrs*

importing needed requered labaries for work


In [1]:
!pip install timm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from __future__ import absolute_import
from __future__  import division
from __future__ import print_function
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms  
from torch.utils.data.sampler import SubsetRandomSampler  
from torch.utils.data import Dataset, DataLoader
from sklearn import model_selection, metrics

import pandas as pd
import torch.nn.functional as F
import os

# import the library and install it

In [4]:
import menovideo.menovideo as menoformer
import menovideo.videopre as vide_reader

**example of using defualt DeVTR without pre-trained wights**
 
the parameters is :

1.   w : is the  path for pre-traied DeVtr model
2.  base : is the base Conv net that work as input embding layer if set to 1.  3.  default will use paper default work with vgg19
4.  classifier :  is the classifier DNN that recive  the output from the 1.  5.  transformer encoder if use default will use same architcture of our main paper
6.  if you want to use your own architcture just pass to it any  nn.sequential  network will work
7.  mid_layer : is the FNN that diractly after the transformer  output
8.  mid_drop : is the dropout for the mid_layer
9.  num_class : is the number of the output class
10.   dim_embd : is the dimionsionalty that we want our input to be  after 1.  11.  transformed by time distrpyted convnet
12.   dr_rate :   is the droupout of the transformer encoder
13.  time_stp : is the number of frames of the input 

 



In [16]:
#not pre-trained
'''
w : is the  path for pre-traied DeVtr model
base : is the base Conv net that work as input embding layer if set to default will use paper default work with vgg19
classifier :  is the classifier DNN that recive  the output from the transformer encoder if use default will use same architcture of our main paper
if you want to use your own architcture just pass to it any  nn.sequential network will work
mid_layer : is the FNN that diractly after the transformer  output
mid_drop : is the dropout for the mid_layer
num_class : is the number of the output class
 dim_embd : is the dimionsionalty that we want our input to be  after transformed by time distrpyted convnet
 dr_rate :   is the droupout of the transformer encoder
time_stp : is the number of frames of the input 

defualt values 
menoformer.DeVTr(w= 'none' , base ='default' ,classifier='default',mid_layer=1024,mid_drop=0.4, num_classes=1 , dim_embd = 512  ,dr_rate= 0.1  , time_stp = 40)
'''
model = menoformer.DeVTr()
model

Sequential(
  (0): TimeWarp(
    (baseModel): Sequential(
      (0): VGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
          (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (9): ReLU(inplace=True)
          (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stat

**example of loading pre-trained DeVTR**

#download the wights from
# https://drive.google.com/file/d/1s7Z1c-4zC522BFVM5EiZDMQLe6ZV8QQh/view?usp=sharing

In [18]:
#pre-trained 
#wight = '/content/drive/MyDrive/Colab Notebooks/vg19bn-40-convtransformer-ep-0.pth'
wight = 'drive/MyDrive/Colab Notebooks/transformers/violance-detaction-myresearch/vg19bn40convtransformer-ep-0.pth'
model2 = menoformer.DeVTr(w= wight , base ='default')
model2

Sequential(
  (0): TimeWarp(
    (baseModel): Sequential(
      (0): VGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
          (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (9): ReLU(inplace=True)
          (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stat

# **use costume Conv net as input embding layer**

In [22]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'cspdarknet53',
 'cspresnet50',
 'cspresnext50',
 'deit_base_distilled_patch16_224',
 'deit_base_distilled_patch16_384',
 'deit_base_patch16_224',
 'deit_base_patch16_384',
 'deit_small_distilled_patch16_224',
 'deit_small_patch16_224',
 'deit_tiny_distilled_patch16_224',
 'deit_tiny_patch16_224',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dm_nfnet_f0',
 'dm_nfnet_f1',
 'dm_nfnet_f2',
 'dm_nfnet_f3',
 'dm_nfnet_f4',
 'dm_nfnet_f5',
 'dm_nfnet_f6',
 'dpn68',
 'dpn

In [25]:
dim_embd = 128
baseModel  =  timm.create_model('resnet26', pretrained=True , num_classes=dim_embd )

model_res = menoformer.DeVTr(base = baseModel , num_classes=10 , dim_embd=dim_embd)
model_res

Sequential(
  (0): TimeWarp(
    (baseModel): Sequential(
      (0): ResNet(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act2): ReLU(inplace=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

# **use costume classifier network**

In [27]:
dim_embd = 128
time_stp = 40
num_classes = 2
mid_layer = 1024
mid_drop = 0.35
classifier  = nn.Sequential(  nn.Linear(time_stp * dim_embd, mid_layer),
                                nn.Dropout(mid_drop),
                                nn.ReLU(),              
            nn.Linear(mid_layer, num_classes)
            )
model_cls = menoformer.DeVTr( classifier= classifier, num_classes=num_classes , dim_embd=dim_embd , time_stp=time_stp)
model_cls

Sequential(
  (0): TimeWarp(
    (baseModel): Sequential(
      (0): VGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
          (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (9): ReLU(inplace=True)
          (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stat

#  **costum classifier and  input embding convnet**

In [28]:
dim_embd = 128
time_stp = 40
num_classes = 1
mid_layer = 1024
mid_drop = 0.35

dim_embd = 128
baseModel  =  timm.create_model('resnet26', pretrained=True , num_classes=dim_embd )

 
classifier  = nn.Sequential(  nn.Linear(time_stp * dim_embd, mid_layer),
                                nn.Dropout(mid_drop),
                                nn.ReLU(),              
            nn.Linear(mid_layer, num_classes)
            )
model_clsres = menoformer.DeVTr(base = baseModel , classifier= classifier, num_classes=num_classes , dim_embd=dim_embd , time_stp=time_stp)
model_clsres

Sequential(
  (0): TimeWarp(
    (baseModel): Sequential(
      (0): ResNet(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act2): ReLU(inplace=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [6]:
print(torch.__version__)

1.9.0+cu111


# using  the helper dataset method of the package and the pre-traiend model on real data from kaggle

In [5]:
! pip install -q kaggle


In [6]:
 from google.colab import files


In [7]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"iraqai","key":"42ec6bd05c8d3eaa132a15c4468d2081"}'}

In [8]:
! mkdir ~/.kaggle

In [9]:
! cp kaggle.json ~/.kaggle/

In [10]:
! chmod 600 ~/.kaggle/kaggle.json

In [11]:

! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              235KB  2021-10-10 17:55:57          13524  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           8121  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           3605  
fatiimaezzahra/famous-iconic-women                          Famous Iconic Women                               838MB  2021-02-28 14:56:00           1550  
dhruvildave/wikibooks-dataset                               Wikibooks Datase

In [12]:
!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset

100% 3.57G/3.58G [00:28<00:00, 77.1MB/s]
100% 3.58G/3.58G [00:28<00:00, 135MB/s] 


In [13]:
!unzip 'real-life-violence-situations-dataset'

Archive:  real-life-violence-situations-dataset.zip
  inflating: Real Life Violence Dataset/NonViolence/NV_1.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_10.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_100.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_1000.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_101.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_102.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_103.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_104.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_105.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_106.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_107.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_108.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_109.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_11.mp4  
  inflating: Real Life Violence Dataset/N

In [5]:
!ls -lia "real life violence situations/Real Life Violence Dataset"

total 80
5373954 drwxr-xr-x 4 root root  4096 Oct 11 21:06 .
5373953 drwxr-xr-x 3 root root  4096 Oct 11 21:06 ..
5373955 drwxr-xr-x 2 root root 36864 Oct 11 21:06 NonViolence
5374956 drwxr-xr-x 2 root root 36864 Oct 11 21:06 Violence


In [6]:

import time
import os
import time
import random
from datetime import datetime

In [7]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(1001)

In [8]:
torch.cuda.empty_cache()
torch.__version__

'1.9.0+cu111'

In [9]:
device =  'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [10]:
time_stp = 40
RGB = 3
H = 200
W = 200

In [11]:
def manual_count(filename):
    i=0
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval , frame = vc.read()
        if frame is None:
            print(filename)
            return False
    else:
        rval = False
    i +=1
    while i < 45:
        rval, frame = vc.read()
        if frame is None:
            print(filename)
            return False
        i +=1

    return True
def path_list(main_dir,filename):
    dk = []
    label = []
    i = 0
    for x in os.listdir(main_dir):
        print(x)
        if 1 == 1:
            td = main_dir+x+'/'
            for file in os.listdir(td):
                #print(td+file)
                if manual_count(td+file):
                    fl = os.path.join(td, file)
                    dk.append(fl)
                    if x == 'Violence':
                        label.append(1)
                    else:
                        label.append(0)
    df = pd.DataFrame(data={"file": dk, "label": label})
    df.to_csv(filename, sep=',',index=False)
    return filename
path_list('real life violence situations/Real Life Violence Dataset/',"./mtrain.csv")

NonViolence
real life violence situations/Real Life Violence Dataset/NonViolence/NV_940.mp4
real life violence situations/Real Life Violence Dataset/NonViolence/NV_354.mp4
Violence


'./mtrain.csv'

In [12]:
df = pd.read_csv('./mtrain.csv')
df.head()

,file,label
0,real life violence situations/Real Life Violen...,0
1,real life violence situations/Real Life Violen...,0
2,real life violence situations/Real Life Violen...,0
3,real life violence situations/Real Life Violen...,0
4,real life violence situations/Real Life Violen...,0


In [13]:
train_df, valid_df = model_selection.train_test_split(
    df, test_size=0.2, random_state=42
)

**using video dataset reader by our package it read the path of video and sample it and return dataset loader contain the video as tensor with it label and the video will be  subtracted to equal the number of frames passed to class**


parameters is :
 

1.   pandas dataframe contain the path and label of each video
2.   number of frames for the  singal video
3.   RGB is the number of color channles
4.   h is the hieght of the frame for each video 
5.   w is the width of the frame for each video



In [14]:
valid_dataset = vide_reader.TaskDataset(valid_df,timesep=time_stp,rgb=RGB,h=H,w=W)

In [15]:
del df
del train_df
del valid_df

In [16]:
BATCH_SIZE = 16
val_BATCH_SIZE = 16
valchecking = 1
epoch = 7
monit = 'val_loss'

In [17]:

valid_loader =    DataLoader(  dataset=valid_dataset,
        batch_size=val_BATCH_SIZE,pin_memory=True,
        
        drop_last=True,
        num_workers=0,shuffle=False)
dataloaders = {  'test':valid_loader}
dataset_sizes = {  'test':len(valid_dataset) }

In [18]:
wight = 'drive/MyDrive/Colab Notebooks/transformers/violance-detaction-myresearch/vg19bn40convtransformer-ep-0.pth'
model = menoformer.DeVTr(w= wight , base ='default')
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
sched =  lr_scheduler.ReduceLROnPlateau(optimizer,mode='min', factor=0.5, patience=2 , verbose=True)
model.eval()

Sequential(
  (0): TimeWarp(
    (baseModel): Sequential(
      (0): VGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
          (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (9): ReLU(inplace=True)
          (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [19]:
def test_model(model, criteria, optimizer, device='cuda'):
    model.to(device)
    since = time.time()
    model.eval()   # Set model to evaluate mode
    running_loss = 0.0
    running_corrects = 0
    pred_vs_real = {}
    pred_vs_real['pred']= []
    pred_vs_real['real']= []
    tk0 = tqdm(dataloaders['test'], total=int(len(dataloaders['test'])))
    counter = 0
    for bi , sample in enumerate(tk0):
        inputs = sample['video'].to(device , dtype=torch.float)
        labels = sample['label'].view(sample['label'].shape[0],1)
        labels = labels.to(device, dtype=torch.float)
        outputs = model(inputs)
        preds = torch.round(torch.sigmoid(outputs))
        pred_vs_real['pred'].append(preds.cpu().detach().numpy())
        pred_vs_real['real'].append(labels.cpu().detach().numpy())
        loss = criterion(outputs, labels)
        # statistics
        counter += 1
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        tk0.set_postfix(loss=(running_loss / (counter * dataloaders['test'].batch_size )))
    epoch_loss = running_loss / dataset_sizes['test']
    epoch_acc = running_corrects.double() / dataset_sizes['test']
    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'test', epoch_loss, epoch_acc))        

    time_elapsed = time.time() - since
    print('testing complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return pred_vs_real 

In [20]:
from tqdm.notebook import tqdm
import time
import json
import copy

In [21]:
rl_vs_prd = test_model(model, criterion, optimizer, device)


  0%|          | 0/25 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


test Loss: 0.0834 Acc: 0.9750
testing complete in 9m 45s
